# Train Various Models on MNIST using kubeflow and seldon-core

Using:

 * kubeflow
 * seldon-core
 
The example will be the MNIST handwriiten digit classification task.

![MNIST](mnist.png "MNIST Digits")

### Dependencies

  * Argo

# Setup


In [1]:
!kubectl config set-context oldcluster --namespace=kubeflow

Context "oldcluster" modified.


# Tensorflow Model
 A simple neural network in Tensorflow.

### Training
 * Create image from source
 * Run training
 

Run with:
  * ``` -p build-push-image=true``` to build image and push to repo, needed extra params:
    * ``` -p version=<version>``` create ```<version>``` of model
    * ``` -p github-user=<github-user>``` to download example-seldon source from ```<github-user>``` account
    * ``` -p github-revision=<revision>``` to use the github branch ```<revision>```
    * ``` -p docker-org=<docker-org>``` to use Docker repo ```<docker-org>``` to push image to. Needs docker credentials in secret as described in README.

In [8]:
!pygmentize ../workflows/training-tf-mnist-workflow.yaml

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: kubeflow-tf-train-
spec:
  entrypoint: workflow
  arguments:
    parameters:
    - name: version
      value: 0.1
    - name: github-user
      value: kubeflow
    - name: github-revision
      value: master
    - name: docker-org
      value: index.docker.io/seldonio
    - name: tfjob-version-hack
      value: 1
    - name: build-push-image
      value: false
  volumes:
  - name: docker-config
    secret:
      secretName: docker-config     # name of an existing k8s secret
  volumeClaimTemplates:
  - metadata:
      name: workspace
    spec:
      accessModes: [ "ReadWriteOnce" ]
      resources:
        requests:
          storage: 0.5Gi
  templates:
  - name: workflow
    steps:
    - - name: get-source
        template: get-source-code
    - - name: build-push
        template: build-and-push
        when: "{{workflow.parameters.build-push-image}} == true"
    - - name: train
        template: tfjob
  - name:

In [12]:
!argo submit --serviceaccount jupyter-notebook ../workflows/training-tf-mnist-workflow.yaml -p tfjob-version-hack=1

Name:                kubeflow-tf-train-ktmjl
Namespace:           kubeflow
ServiceAccount:      tf-job-operator
Status:              Pending
Created:             Wed Jun 26 13:21:12 +0200 (1 second ago)
Parameters:          
  tfjob-version-hack: 1
  version:           0.1
  github-user:       kubeflow
  github-revision:   master
  docker-org:        index.docker.io/seldonio
  build-push-image:  false


In [6]:
!argo list

ERROR: logging before flag.Parse: E0626 12:47:36.549017   36926 request.go:853] Unexpected error when reading response body: &net.OpError{Op:"read", Net:"tcp", Source:(*net.TCPAddr)(0xc0004940f0), Addr:(*net.TCPAddr)(0xc000494120), Err:(*os.SyscallError)(0xc0002e8300)}
2019/06/26 12:47:36 Unexpected error &net.OpError{Op:"read", Net:"tcp", Source:(*net.TCPAddr)(0xc0004940f0), Addr:(*net.TCPAddr)(0xc000494120), Err:(*os.SyscallError)(0xc0002e8300)} when reading response body. Please retry.


### Runtime Image

Run with:
  * ``` -p build-push-image=true``` to build image and push to repo, needed extra params:
    * ``` -p version=<version>``` create ```<version>``` of model
    * ``` -p github-user=<github-user>``` to download example-seldon source from ```<github-user>``` account
    * ``` -p github-revision=<revision>``` to use the github branch ```<revision>```
    * ``` -p docker-org=<docker-org>``` to use Docker user ```<docker-org>``` to push image to. Needs docker credentials in secret as described in README.
  * ``` -p deploy-model=true``` to deploy model

In [ ]:
!pygmentize ../workflows/serving-tf-mnist-workflow.yaml

In [ ]:
!argo submit ../workflows/serving-tf-mnist-workflow.yaml

In [ ]:
!argo list

# Sklearn Model
A Random forest in sklearn.

### Training

 * For options see above Tensorflow example

In [ ]:
!pygmentize ../workflows/training-sk-mnist-workflow.yaml

In [ ]:
!argo submit ../workflows/training-sk-mnist-workflow.yaml

In [ ]:
!argo list

### Runtime Image
 * For options see above Tensorflow example

In [ ]:
!pygmentize ../workflows/serving-sk-mnist-workflow.yaml

In [ ]:
!argo submit ../workflows/serving-sk-mnist-workflow.yaml

In [ ]:
!argo list

# R Model
A partial least squares model in R.

### Training

 * For options see above Tensorflow example

In [ ]:
!pygmentize ../workflows/training-r-mnist-workflow.yaml

In [ ]:
!argo submit ../workflows/training-r-mnist-workflow.yaml

In [ ]:
!argo list

### Runtime Image
 * For options see above Tensorflow example

In [ ]:
!pygmentize ../workflows/serving-r-mnist-workflow.yaml

In [ ]:
!argo submit ../workflows/serving-r-mnist-workflow.yaml

In [ ]:
!argo list